In [ ]:
import pandas as pd
with open('./product_train_0.txt', 'r', encoding='utf-8') as file:
    data = file.read().split('\n')
    raw_df = pd.DataFrame([line.split(' ', 1) for line in data], columns=['Label', 'Description'])
raw_df = raw_df.drop(raw_df.index[-1])
raw_df['Label'] = raw_df['Label'].str.replace('__label__', '').str.replace('_', ' ')
labels = raw_df['Label'].unique()
label_counts = raw_df['Label'].value_counts()
minor_labels = label_counts[label_counts < 8].index.tolist()
label_count = len(minor_labels)

In [ ]:
# try:
#     crawl_res = pd.read_csv('crawl_res_2.csv')
# except:
#     crawl_res = pd.DataFrame(columns=['Label', 'Description'])
# crawled_list = crawl_res['Label'].unique().tolist()
# labels = list(set(labels)-set(crawled_list))
# len(labels)

In [ ]:
try:
    crawl_res = pd.read_csv('crawl_res_4.csv')
except:
    crawl_res = pd.DataFrame(columns=['Label', 'Description'])
crawled_list = crawl_res['Label'].unique().tolist()
labels_to_crawl = list(set(minor_labels)-set(crawled_list))
len(labels_to_crawl)

1505

In [1]:
import requests
import urllib.parse
import threading
from IPython.display import clear_output

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
}

data_lock = threading.Lock()
def tiki_crawl(label):
    global crawl_res
    global labels_to_crawl
    global crawled_list
    # Encode the key word as a URL query parameter
    search_keyword = urllib.parse.quote(label, encoding='utf-8')
    # Set limit and page for the api fetching
    page = 1
    limit = 50
    new_data = pd.DataFrame(columns=['Label', 'Description'])
    while not label in crawled_list:
        # Send an HTTP GET request to Tiki.com with the search keyword
        url = f"https://tiki.vn/api/v2/products?limit={limit}&q={search_keyword}&page={page}"
        response = requests.get(url, headers=headers)
        # Check if the request was successful
        if response.status_code == 200:
            # Access response data
            data = response.json()['data']
            if data == [] or page == 3:
                break
            # Print the list of product names
            for product in data:
                new_data.loc[len(new_data), ['Label', 'Description']] = label, product['name']
            page += 1
        else:
            print(f"Failed to retrieve the search results with {label}. {len(crawled_list)}/{len(labels_to_crawl)}")
            break
    with data_lock:
        crawl_res = pd.concat([crawl_res, new_data], ignore_index=True)
        crawl_res.to_csv('crawl_res_2.csv', index=False)
    crawled_list.append(label)
    crawled_list = list(dict.fromkeys(crawled_list))
    clear_output(wait=True)
    print(f"Finish crawling label {label}. {len(crawled_list)}/{label_count}")


In [ ]:
# from concurrent.futures import ThreadPoolExecutor

# num_threads = 3

# with ThreadPoolExecutor(max_workers=num_threads) as executor:
#     for label in labels_to_crawl:
#         executor.submit(tiki_crawl, label)


In [ ]:
def vatgia_crawl(label):
    global crawl_res
    global labels_to_crawl
    global crawled_list
    # Encode the key word as a URL query parameter
    search_keyword = urllib.parse.quote(label, encoding='utf-8')
    # Set limit and page for the api fetching
    page = 1
    new_data = pd.DataFrame(columns=['Label', 'Description'])
    while not label in crawled_list:
        # Send an HTTP GET request to Tiki.com with the search keyword
        url = f"https://vatgia.com/api/v2/raovat/search_result?page={page}&key={search_keyword}"
        response = requests.get(url, headers=headers)
        # Check if the request was successful
        if response.status_code == 200:
            # Access response data
            data = response.json()['data']['products']
            if data == [] or page == 3:
                break
            # Print the list of product names
            for product in data:
                if product['classify'] == 'normal':
                    new_data.loc[len(new_data), ['Label', 'Description']] = label, product['name']
            page += 1
        else:
            print(f"Failed to retrieve the search results with {label}. {len(crawled_list)}/{len(labels_to_crawl)}")
            break
    with data_lock:
        crawl_res = pd.concat([crawl_res, new_data], ignore_index=True)
        crawl_res.to_csv('crawl_res_4.csv', index=False)
    crawled_list.append(label)
    crawled_list = list(dict.fromkeys(crawled_list))
    clear_output(wait=True)
    print(f"Finish crawling label {label}. {len(crawled_list)}/{label_count}")

In [ ]:
from concurrent.futures import ThreadPoolExecutor

num_threads = 3

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    for label in labels_to_crawl:
        executor.submit(vatgia_crawl, label)


Finish crawling label Tour du lịch Châu Á. 1505/1505
